In [1]:
import numpy as np
import pandas as pd

In [2]:
state2abbr = {"Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR", "California": "CA",
    "Colorado": "CO", "Connecticut": "CT", "Delaware": "DE", "Florida": "FL", "Georgia": "GA", "Hawaii": "HI",
    "Idaho": "ID", "Illinois": "IL", "Indiana": "IN", "Iowa": "IA", "Kansas": "KS", "Kentucky": "KY",
    "Louisiana": "LA", "Maine": "ME", "Maryland": "MD", "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN",
    "Mississippi": "MS", "Missouri": "MO", "Montana": "MT", "Nebraska": "NE", "Nevada": "NV", "New Hampshire": "NH",
    "New Jersey": "NJ", "New Mexico": "NM", "New York": "NY", "North Carolina": "NC", "North Dakota": "ND", "Ohio": "OH",
    "Oklahoma": "OK", "Oregon": "OR", "Pennsylvania": "PA", "Rhode Island": "RI", "South Carolina": "SC", "South Dakota": "SD",
    "Tennessee": "TN", "Texas": "TX", "Utah": "UT", "Vermont": "VT", "Virginia": "VA", "Washington": "WA", "West Virginia": "WV",
    "Wisconsin": "WI", "Wyoming": "WY", "District of Columbia": "DC",
}
    
# invert the dictionary
abbr2state = dict(map(reversed, state2abbr.items()))
state_list = list(state2abbr.keys())
state_list.pop(1)

'Alaska'

# Census Data

In [3]:
def weight_average_column(df, col_name):
    data = df[["State", "County", "TotalPop", col_name]]
    data["tmp"] = data["TotalPop"] * data[col_name]
    agg_data = data.groupby(["State", "County"], as_index = False).agg("sum")
    agg_data[col_name] = agg_data["tmp"] / agg_data["TotalPop"]
    return agg_data[["State", "County", col_name]]

def load_clean_census_data():
    census_var_list = ["State", "County", "TotalPop", "Men", "Women"]
    census_weight_list = ["Hispanic", "White", "Black", "Native", "Asian", "Pacific", "Drive", "Carpool", "Transit", "Walk", "OtherTransp", "WorkAtHome", "MeanCommute"]
    full_data = pd.read_csv("../datasets/acs2017_census_tract_data.csv")
    full_data = full_data.dropna(how = "all")
    full_data = full_data[census_var_list + census_weight_list]
    basic_df = full_data[census_var_list]
    agg_data = basic_df.groupby(["State", "County"], as_index = False).agg("sum")
    for weight_name in census_weight_list:
        column = weight_average_column(full_data, weight_name)
        agg_data = pd.merge(agg_data, column, on=["State", 'County'])
    agg_data = agg_data[agg_data["State"] != 'Puerto Rico']
    agg_data["County"] = agg_data["County"].apply(lambda s: s.split(" ")[0] if len(s.split(" ")) == 2 else s.split(" ")[0] + " " + s.split(" ")[1])
    return agg_data

In [4]:
census_df = load_clean_census_data()
census_df

<ipython-input-3-a2df968c33a0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["tmp"] = data["TotalPop"] * data[col_name]


,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute
0,Alabama,Autauga,55036,26899,28137,2.672443,75.422703,18.932544,0.322632,0.939085,0.024602,85.874886,9.538171,0.060964,0.586184,1.379277,2.543982,25.899355
1,Alabama,Baldwin,203360,99527,103833,4.445085,83.080754,9.462884,0.752043,0.682431,0.000000,84.534211,7.777040,0.110775,0.810847,1.161533,5.627419,26.951154
2,Alabama,Barbour,26201,13976,12225,4.202309,45.736300,47.829587,0.175959,0.578573,0.000000,83.243827,11.156055,0.298237,2.365803,1.693775,1.244479,23.968818
3,Alabama,Bibb,22580,12251,10329,2.337236,74.634783,21.996639,0.382334,0.000000,0.000000,86.367228,9.433627,0.629880,0.314628,1.854570,1.438109,29.867551
4,Alabama,Blount,57667,28490,29177,9.000794,87.358163,1.527107,0.341069,0.153852,0.000000,86.841570,10.123662,0.131966,0.415479,0.370345,2.116228,35.106992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,Wyoming,Sweetwater,44527,22981,21546,16.007211,79.631911,0.774173,0.559919,0.579363,0.461641,77.371013,14.416172,2.582907,2.883904,1.290639,1.458735,20.481366
3216,Wyoming,Teton,22923,12169,10754,14.977054,81.521943,0.463574,0.256825,2.190944,0.000000,68.318008,6.679649,3.934843,11.555333,3.827496,5.696990,14.263569
3217,Wyoming,Uinta,20758,10593,10165,9.069024,87.644397,0.134531,0.940423,0.110964,0.000000,77.370320,14.908642,3.383390,1.098203,1.323225,2.022931,19.983414
3218,Wyoming,Washakie,8253,4118,4135,14.244820,82.212529,0.310384,0.354441,0.126257,0.000000,77.088198,10.307658,0.031564,6.916552,1.359700,4.296329,14.774930


# Economic Data

In [5]:
def extract_feature(full_df, feature_name, col_name):
    df = full_df[(full_df["Description"] == feature_name) & (full_df["2018"] != '(NA)')]
    df = df[["GeoFIPS", "GeoName", "2018"]]
    df["2018"] = df["2018"].astype(int)
    df = df.rename(columns={"2018":col_name})
    return df

def load_clean_economic_data():
    raw_data = pd.read_csv("../datasets/economic_profile_raw.csv", encoding='latin-1')
    raw_data = raw_data.dropna(how="all")
    df_data = raw_data[["GeoFIPS", "GeoName", "Description", "Unit", "2018"]]
    df_data = df_data[df_data["GeoFIPS"].str.len() == 8]
    df_data["GeoFIPS"] = df_data["GeoFIPS"].apply(lambda x: x[2:-1])
    df_county = df_data[~df_data["GeoFIPS"].str.endswith("000")]
    county_income_df = extract_feature(df_county, " Per capita personal income 4/", "income")
    county_employments_df = extract_feature(df_county, "Total employment (number of jobs)", "totalEmploy")
    county_salary_df = extract_feature(df_county, " Average wages and salaries", "salary")
    agg_df = county_income_df
    agg_df = pd.merge(agg_df, county_employments_df, on=["GeoFIPS", "GeoName"])
    agg_df = pd.merge(agg_df, county_salary_df, on=["GeoFIPS", "GeoName"])
    agg_df["State"] = agg_df["GeoName"].apply(lambda s: abbr2state[s[-3:-1] if s[-1] == "*" else s[-2:]])
    agg_df["County"] = agg_df["GeoName"].apply(lambda s: s.split(",")[0])
    agg_df["County"] = agg_df["County"].apply(lambda s: s.split(" ")[0] if len(s.split(" ")) == 1 else s.split(" ")[0] + " " + s.split(" ")[1])                                           
    agg_df = agg_df.drop(columns=["GeoName"])
    return agg_df

economic_df = load_clean_economic_data()

/Users/liboxuan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (2,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [6]:
join_data = pd.merge(census_df, economic_df, on=["State", "County"])

In [7]:
join_data

,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,...,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,GeoFIPS,income,totalEmploy,salary
0,Alabama,Autauga,55036,26899,28137,2.672443,75.422703,18.932544,0.322632,0.939085,...,9.538171,0.060964,0.586184,1.379277,2.543982,25.899355,01001,42504,18172,38739
1,Alabama,Baldwin,203360,99527,103833,4.445085,83.080754,9.462884,0.752043,0.682431,...,7.777040,0.110775,0.810847,1.161533,5.627419,26.951154,01003,46205,115748,37428
2,Alabama,Barbour,26201,13976,12225,4.202309,45.736300,47.829587,0.175959,0.578573,...,11.156055,0.298237,2.365803,1.693775,1.244479,23.968818,01005,35067,11506,38032
3,Alabama,Bibb,22580,12251,10329,2.337236,74.634783,21.996639,0.382334,0.000000,...,9.433627,0.629880,0.314628,1.854570,1.438109,29.867551,01007,30559,6454,40495
4,Alabama,Blount,57667,28490,29177,9.000794,87.358163,1.527107,0.341069,0.153852,...,10.123662,0.131966,0.415479,0.370345,2.116228,35.106992,01009,35671,15702,35305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3079,Wyoming,Sweetwater,44527,22981,21546,16.007211,79.631911,0.774173,0.559919,0.579363,...,14.416172,2.582907,2.883904,1.290639,1.458735,20.481366,56037,55352,27908,60194
3080,Wyoming,Teton,22923,12169,10754,14.977054,81.521943,0.463574,0.256825,2.190944,...,6.679649,3.934843,11.555333,3.827496,5.696990,14.263569,56039,230141,33629,50939
3081,Wyoming,Uinta,20758,10593,10165,9.069024,87.644397,0.134531,0.940423,0.110964,...,14.908642,3.383390,1.098203,1.323225,2.022931,19.983414,56041,40425,11806,42424
3082,Wyoming,Washakie,8253,4118,4135,14.244820,82.212529,0.310384,0.354441,0.126257,...,10.307658,0.031564,6.916552,1.359700,4.296329,14.774930,56043,49310,5255,42110


# Housing Price Data

In [8]:
housing_df = pd.read_csv("../datasets/2021_median_home_prices_raw.csv", sep='\t', encoding = "utf-16")

In [9]:
housing_df = housing_df[["Geography", "Q2 Price"]]
housing_df.columns = ["County", "Housing Price"]
housing_df["State"] = housing_df["County"].apply(lambda s : s.split(",")[-1].strip())
housing_df["County"] = housing_df["County"].apply(lambda s : s.split(",")[0].split(" County")[0].strip())
housing_df

,County,Housing Price,State
0,Autauga,145148.0,Alabama
1,Baldwin,180978.0,Alabama
2,Barbour,95252.0,Alabama
3,Bibb,112839.0,Alabama
4,Blount,131627.0,Alabama
...,...,...,...
3136,Teton,747214.0,Wyoming
3137,Uinta,191629.0,Wyoming
3138,Washakie,174386.0,Wyoming
3139,Weston,193256.0,Wyoming


In [10]:
join_data = pd.merge(join_data, housing_df, on=["State", "County"])

In [11]:
join_data

,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,...,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,GeoFIPS,income,totalEmploy,salary,Housing Price
0,Alabama,Autauga,55036,26899,28137,2.672443,75.422703,18.932544,0.322632,0.939085,...,0.060964,0.586184,1.379277,2.543982,25.899355,01001,42504,18172,38739,145148.0
1,Alabama,Baldwin,203360,99527,103833,4.445085,83.080754,9.462884,0.752043,0.682431,...,0.110775,0.810847,1.161533,5.627419,26.951154,01003,46205,115748,37428,180978.0
2,Alabama,Barbour,26201,13976,12225,4.202309,45.736300,47.829587,0.175959,0.578573,...,0.298237,2.365803,1.693775,1.244479,23.968818,01005,35067,11506,38032,95252.0
3,Alabama,Bibb,22580,12251,10329,2.337236,74.634783,21.996639,0.382334,0.000000,...,0.629880,0.314628,1.854570,1.438109,29.867551,01007,30559,6454,40495,112839.0
4,Alabama,Blount,57667,28490,29177,9.000794,87.358163,1.527107,0.341069,0.153852,...,0.131966,0.415479,0.370345,2.116228,35.106992,01009,35671,15702,35305,131627.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2981,Wyoming,Sweetwater,44527,22981,21546,16.007211,79.631911,0.774173,0.559919,0.579363,...,2.582907,2.883904,1.290639,1.458735,20.481366,56037,55352,27908,60194,207029.0
2982,Wyoming,Teton,22923,12169,10754,14.977054,81.521943,0.463574,0.256825,2.190944,...,3.934843,11.555333,3.827496,5.696990,14.263569,56039,230141,33629,50939,747214.0
2983,Wyoming,Uinta,20758,10593,10165,9.069024,87.644397,0.134531,0.940423,0.110964,...,3.383390,1.098203,1.323225,2.022931,19.983414,56041,40425,11806,42424,191629.0
2984,Wyoming,Washakie,8253,4118,4135,14.244820,82.212529,0.310384,0.354441,0.126257,...,0.031564,6.916552,1.359700,4.296329,14.774930,56043,49310,5255,42110,174386.0


# Climate Data

In [12]:
climate_df = pd.read_csv("../datasets/climate/cleaned/climate.csv")

In [13]:
climate_df = climate_df[climate_df["Year"] == 2019]
climate_df["State"] = climate_df["State"].apply(lambda s: abbr2state[s])
climate_df["County"] = climate_df["County"].apply(lambda s: s.split(" County")[0].strip())
climate_df = climate_df[["State", "County", "Temp Avg", "Temp Max", "Temp Min", "Precipitation"]]

In [14]:
join_data = pd.merge(join_data, climate_df, on=["State", "County"])
join_data

,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,...,MeanCommute,GeoFIPS,income,totalEmploy,salary,Housing Price,Temp Avg,Temp Max,Temp Min,Precipitation
0,Alabama,Autauga,55036,26899,28137,2.672443,75.422703,18.932544,0.322632,0.939085,...,25.899355,01001,42504,18172,38739,145148.0,65.7,76.6,54.7,70.98
1,Alabama,Baldwin,203360,99527,103833,4.445085,83.080754,9.462884,0.752043,0.682431,...,26.951154,01003,46205,115748,37428,180978.0,68.6,78.3,58.8,75.80
2,Alabama,Barbour,26201,13976,12225,4.202309,45.736300,47.829587,0.175959,0.578573,...,23.968818,01005,35067,11506,38032,95252.0,66.3,76.9,55.7,64.46
3,Alabama,Bibb,22580,12251,10329,2.337236,74.634783,21.996639,0.382334,0.000000,...,29.867551,01007,30559,6454,40495,112839.0,64.4,75.4,53.3,68.09
4,Alabama,Blount,57667,28490,29177,9.000794,87.358163,1.527107,0.341069,0.153852,...,35.106992,01009,35671,15702,35305,131627.0,63.1,72.8,53.3,66.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2977,Wyoming,Sweetwater,44527,22981,21546,16.007211,79.631911,0.774173,0.559919,0.579363,...,20.481366,56037,55352,27908,60194,207029.0,42.8,56.3,29.4,8.76
2978,Wyoming,Teton,22923,12169,10754,14.977054,81.521943,0.463574,0.256825,2.190944,...,14.263569,56039,230141,33629,50939,747214.0,35.6,47.6,23.7,34.90
2979,Wyoming,Uinta,20758,10593,10165,9.069024,87.644397,0.134531,0.940423,0.110964,...,19.983414,56041,40425,11806,42424,191629.0,40.7,54.0,27.3,10.87
2980,Wyoming,Washakie,8253,4118,4135,14.244820,82.212529,0.310384,0.354441,0.126257,...,14.774930,56043,49310,5255,42110,174386.0,43.9,57.2,30.6,11.92


# Education Data

In [15]:
df_education = pd.read_csv("../datasets/education_raw.csv", thousands=',')
df_education = df_education.dropna(how = "all")
df_education = df_education[["State", "Area name", "Percent of adults with a bachelor's degree or higher, 2015-19"]]
df_education.columns = ["State", "County", "Advanced Degree Percentage"]
df_education = df_education[~df_education["State"].isin(["PR", "US"])]
df_education["State"] = df_education["State"].apply(lambda s: abbr2state[s])
df_education["County"] = df_education["County"].apply(lambda s: s.split(" County")[0].strip())
df_education

,State,County,Advanced Degree Percentage
0,Alabama,Bibb,10.4
1,Alabama,Blount,13.1
2,Alabama,Chilton,12.7
3,Alabama,Jefferson,33.4
4,Alabama,St. Clair,16.1
...,...,...,...
3277,Virginia,Clifton Forge city,NaN
3278,Washington,Washington,36.0
3279,West Virginia,West Virginia,20.6
3280,Wisconsin,Wisconsin,30.1


In [16]:
join_data = pd.merge(join_data, df_education, on=["State", "County"])
join_data = join_data.drop_duplicates(subset=["State", "County"])
join_data

,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,...,GeoFIPS,income,totalEmploy,salary,Housing Price,Temp Avg,Temp Max,Temp Min,Precipitation,Advanced Degree Percentage
0,Alabama,Autauga,55036,26899,28137,2.672443,75.422703,18.932544,0.322632,0.939085,...,01001,42504,18172,38739,145148.0,65.7,76.6,54.7,70.98,26.6
1,Alabama,Baldwin,203360,99527,103833,4.445085,83.080754,9.462884,0.752043,0.682431,...,01003,46205,115748,37428,180978.0,68.6,78.3,58.8,75.80,31.9
2,Alabama,Barbour,26201,13976,12225,4.202309,45.736300,47.829587,0.175959,0.578573,...,01005,35067,11506,38032,95252.0,66.3,76.9,55.7,64.46,11.6
3,Alabama,Bibb,22580,12251,10329,2.337236,74.634783,21.996639,0.382334,0.000000,...,01007,30559,6454,40495,112839.0,64.4,75.4,53.3,68.09,10.4
4,Alabama,Blount,57667,28490,29177,9.000794,87.358163,1.527107,0.341069,0.153852,...,01009,35671,15702,35305,131627.0,63.1,72.8,53.3,66.56,13.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2982,Wyoming,Sweetwater,44527,22981,21546,16.007211,79.631911,0.774173,0.559919,0.579363,...,56037,55352,27908,60194,207029.0,42.8,56.3,29.4,8.76,22.5
2983,Wyoming,Teton,22923,12169,10754,14.977054,81.521943,0.463574,0.256825,2.190944,...,56039,230141,33629,50939,747214.0,35.6,47.6,23.7,34.90,57.0
2984,Wyoming,Uinta,20758,10593,10165,9.069024,87.644397,0.134531,0.940423,0.110964,...,56041,40425,11806,42424,191629.0,40.7,54.0,27.3,10.87,16.0
2985,Wyoming,Washakie,8253,4118,4135,14.244820,82.212529,0.310384,0.354441,0.126257,...,56043,49310,5255,42110,174386.0,43.9,57.2,30.6,11.92,23.4


In [17]:
join_data["Men"] = join_data["Men"] / join_data["TotalPop"]
join_data["Women"] = join_data["Women"] / join_data["TotalPop"]
join_data["totalEmploy"] = join_data["totalEmploy"] / join_data["TotalPop"]
join_data.columns

Index(['State', 'County', 'TotalPop', 'Men', 'Women', 'Hispanic', 'White',
       'Black', 'Native', 'Asian', 'Pacific', 'Drive', 'Carpool', 'Transit',
       'Walk', 'OtherTransp', 'WorkAtHome', 'MeanCommute', 'GeoFIPS', 'income',
       'totalEmploy', 'salary', 'Housing Price', 'Temp Avg', 'Temp Max',
       'Temp Min', 'Precipitation', 'Advanced Degree Percentage'],
      dtype='object')

In [18]:
join_data.columns = ['State', 'County', 'Total Population', 'Men Percentage', 'Women Percentage', 'Hispanic Percentage', 'White Percentage',
       'Black Percentage', 'Native Percentage', 'Asian Percentage', 'Pacific Percentage', 'Drive', 'Carpool', 'Transit',
       'Walk', 'OtherTransp', 'WorkAtHome', 'MeanCommute', 'GeoFIPS', 'income',
       'Total Employment Rate', 'Salary', 'Housing Price', 'Temperature Avg', 'Temperature Max',
       'Temperature Min', 'Precipitation', 'Advanced Degree Percentage']
join_data

,State,County,Total Population,Men Percentage,Women Percentage,Hispanic Percentage,White Percentage,Black Percentage,Native Percentage,Asian Percentage,...,GeoFIPS,income,Total Employment Rate,Salary,Housing Price,Temperature Avg,Temperature Max,Temperature Min,Precipitation,Advanced Degree Percentage
0,Alabama,Autauga,55036,0.488753,0.511247,2.672443,75.422703,18.932544,0.322632,0.939085,...,01001,42504,0.330184,38739,145148.0,65.7,76.6,54.7,70.98,26.6
1,Alabama,Baldwin,203360,0.489413,0.510587,4.445085,83.080754,9.462884,0.752043,0.682431,...,01003,46205,0.569178,37428,180978.0,68.6,78.3,58.8,75.80,31.9
2,Alabama,Barbour,26201,0.533415,0.466585,4.202309,45.736300,47.829587,0.175959,0.578573,...,01005,35067,0.439144,38032,95252.0,66.3,76.9,55.7,64.46,11.6
3,Alabama,Bibb,22580,0.542560,0.457440,2.337236,74.634783,21.996639,0.382334,0.000000,...,01007,30559,0.285828,40495,112839.0,64.4,75.4,53.3,68.09,10.4
4,Alabama,Blount,57667,0.494043,0.505957,9.000794,87.358163,1.527107,0.341069,0.153852,...,01009,35671,0.272287,35305,131627.0,63.1,72.8,53.3,66.56,13.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2982,Wyoming,Sweetwater,44527,0.516114,0.483886,16.007211,79.631911,0.774173,0.559919,0.579363,...,56037,55352,0.626766,60194,207029.0,42.8,56.3,29.4,8.76,22.5
2983,Wyoming,Teton,22923,0.530864,0.469136,14.977054,81.521943,0.463574,0.256825,2.190944,...,56039,230141,1.467042,50939,747214.0,35.6,47.6,23.7,34.90,57.0
2984,Wyoming,Uinta,20758,0.510309,0.489691,9.069024,87.644397,0.134531,0.940423,0.110964,...,56041,40425,0.568745,42424,191629.0,40.7,54.0,27.3,10.87,16.0
2985,Wyoming,Washakie,8253,0.498970,0.501030,14.244820,82.212529,0.310384,0.354441,0.126257,...,56043,49310,0.636738,42110,174386.0,43.9,57.2,30.6,11.92,23.4


In [19]:
join_data.to_csv("../datasets/merged.csv")